# Analyse des causes d'attrition chez TechNova Partners
## Étape 3 : Feature Engineering - Préparation des données pour la modélisation

**Objectif** : Préparer les données (X et y) pour l'entraînement de modèles de Machine Learning

## 1. Importation des bibliothèques

In [1]:
# 1. Importation des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import shap

#bibliothèque Python
from sklearn.preprocessing import OneHotEncoder

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Bibliothèques importées avec succès")

ModuleNotFoundError: No module named 'shap'

## 2. Chargement du Dataset de l'analyse Exploratoire.

In [ ]:
# 2. Chargement des données complètes
df = pd.read_csv('data_complete.csv')

print(f"📊 Dataset chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.info()

#### 2.1 Affichage de variables quantitatives et qualitatives

In [ ]:
variables_quantitatives =df.select_dtypes(include=['int64', 'float64']).columns.tolist()
variables_qualitatives = df.select_dtypes(include=['object']).columns.tolist()

print("🔢 VARIABLES QUANTITATIVES (numériques) :\n")
for i, col in enumerate(variables_quantitatives, 1):
    print(f"{i:2d}. {col}")

print(f"\n📝 VARIABLES QUALITATIVES (catégorielles) :\n")
for i, col in enumerate(variables_qualitatives, 1):
    print(f"{i:2d}. {col}")

## 3 .Encoding des variables qualitative , methode : OneHotEncoder

### 3.1 Analyse des variables catégorielles

In [ ]:
print("="*80)
print("ÉTAPE 1 : ANALYSE DES VARIABLES CATÉGORIELLES")
print("="*80)

# Lister toutes les variables catégorielles (object)
variables_qualitatives = df.select_dtypes(include=['object']).columns.tolist()

print(f"\n📝 Variables catégorielles détectées : {len(variables_qualitatives)}\n")

# Analyser chaque variable
for col in variables_qualitatives:
    print(f"\n{'='*70}")
    print(f"📌 Variable : {col}")
    print(f"{'='*70}")
    
    # Valeurs uniques
    valeurs_uniques = df[col].dropna().unique()
    nb_valeurs = len(valeurs_uniques)
    
    print(f"Nombre de valeurs uniques : {nb_valeurs}")
    print(f"Valeurs : {list(valeurs_uniques)}")
    
    # Distribution
    print(f"\nDistribution :")
    distribution = df[col].value_counts()
    for val, count in distribution.items():
        pourcentage = (count / len(df)) * 100
        print(f"  - {val:30s} : {count:4d} ({pourcentage:5.2f}%)")
    
    # Valeurs manquantes
    nb_missing = df[col].isnull().sum()
    if nb_missing > 0:
        print(f"\n⚠️  Valeurs manquantes : {nb_missing} ({(nb_missing/len(df)*100):.2f}%)")
    else :
        print("pas de valeurs manquantes sur l'ensemble de Dataset")

---

On a 8 variables catégorielles.  
On observe aucune valeur aberrante.

---

### 3.2 Encodage des variables catégorielles

#### a/ encodage de la variable 'ordre_deplacement' en OrdinaleEncoder

In [ ]:
print("\n" + "="*80)
print("ÉTAPE 2.2 : ENCODAGE DES VARIABLES ORDINALES")
print("="*80)

from sklearn.preprocessing import OrdinalEncoder

# Définir les ordres logiques pour chaque variable ordinale

# 1. Fréquence de déplacement
ordre_deplacement = ['Aucun', 'Occasionnel', 'Frequent']

print("\n📊 Variables ordinales à encoder :\n")
print("1. frequence_deplacement")
print(f"   Ordre : {' < '.join(ordre_deplacement)}")
print(f"   Encodage : {dict(zip(ordre_deplacement, range(len(ordre_deplacement))))}")


# Vérifier que toutes les valeurs sont bien présentes
print("\n🔍 Vérification des valeurs :")
print(f"\nfrequence_deplacement - Valeurs dans les données :")
print(df['frequence_deplacement'].unique())


# Encoder fréquence de déplacement
encoder_deplacement = OrdinalEncoder(categories=[ordre_deplacement])
df['frequence_deplacement_encoded'] = encoder_deplacement.fit_transform(
    df[['frequence_deplacement']]
).astype(int)


print("\n✅ Encodage ordinal terminé !")

#### b/ Encodage simple
Pour les variables qui ont 2 catégories :
- genre
- heures supplémentaire
- a_quitte_l_entreprise (variable cible)

In [ ]:
# ============================================================================
# ÉTAPE 2.4 : ENCODAGE DES VARIABLES BINAIRES
# ============================================================================

print("\n" + "="*80)
print("ÉTAPE 2.4 : ENCODAGE DES VARIABLES BINAIRES")
print("="*80)

# 1. heure_supplementaires : Oui → 1, Non → 0
df['heure_supplementaires_encoded'] = (df['heure_supplementaires'] == 'Oui').astype(int)
df['target'] = (df['a_quitte_l_entreprise'] == 'Oui').astype(int)
df['genre_encoded'] = (df['genre'] == 'M').astype(int)

print("\n✅ Variables binaires encodées :")
print(df[['heure_supplementaires', 'heure_supplementaires_encoded']].value_counts().sort_index())
print(df[['a_quitte_l_entreprise', 'target']].value_counts().sort_index())
print(df[['genre', 'genre_encoded']].value_counts().sort_index())

#### b/ Encodage des autres variables OneHotEncoder

In [ ]:
print("\n" + "="*80)
print("ÉTAPE 2.3 : ENCODAGE DES VARIABLES NOMINALES")
print("="*80)

# Variables nominales à encoder
variables_nominales = [
    'statut_marital',
    'departement',
    'poste',
    'domaine_etude'
]

print(f"\n📝 Variables nominales à encoder : {len(variables_nominales)}\n")
for var in variables_nominales:
    nb_categories = df[var].nunique()
    print(f"  - {var:25s} : {nb_categories} catégories")

# Créer l'encoder avec drop='first' pour éviter la multicolinéarité
encoder_nominal = OneHotEncoder(drop='first', sparse_output=False)

# Encoder
encoded_array = encoder_nominal.fit_transform(df[variables_nominales])

# Récupérer les noms des colonnes
feature_names = encoder_nominal.get_feature_names_out(variables_nominales)

# Créer un DataFrame avec les variables encodées
df_encoded = pd.DataFrame(
    encoded_array, 
    columns=feature_names,
    index=df.index
)

print(f"\n✅ OneHotEncoding terminé !")
print(f"   Nombre de nouvelles colonnes créées : {df_encoded.shape[1]}")
print(f"\n📋 Aperçu des colonnes créées :")
for i, col in enumerate(df_encoded.columns, 1):
    print(f"  {i:2d}. {col}")

## 5. Création de la variable cible

In [ ]:
y = (df['target'])
reste = (y == 0).sum()
parti = (y == 1).sum()
pct_parti= parti/len(y)*100
pct_reste = reste/len(y)*100

print("\n✅ Variable cible 'y' créée avec succès")
print(f"\nType : {type(y)}")
print(f"Shape : {y.shape}")
print(f"\n📊 Distribution de y :")
print(f"  - Restés (0) : {pct_reste:.2f}%")
print(f"  - Partis (1) : {pct_parti:.2f}%")

---

On voit qu'il y a une disparité entre les valeurs de la Target avec une prépondérance pour "restés"

---

## 6. Analyse des variables QUANTITATIVES

### 6.1. Exclusion des features non utilisés

In [ ]:
print("="*80)
print("ÉTAPE 3 : CRÉATION DU DATAFRAME X")
print("="*80)

# 1. Lister les colonnes à EXCLURE de X
colonnes_a_exclure = [
    'id_employee',                          # Identifiant (pas prédictif)
    'a_quitte_l_entreprise',                # Variable cible
    
    # Variables catégorielles ORIGINALES (avant encodage)
    'genre',
    'statut_marital',
    'departement',
    'poste',
    'domaine_etude',
    'heure_supplementaires',
    'frequence_deplacement',

    #la target
    'target',
]


print(f"\n❌ Colonnes à EXCLURE de X ({len(colonnes_a_exclure)}) :")
for col in colonnes_a_exclure:
    print(f"   - {col}")

# 2. Créer X en concaténant :
#    - Variables numériques (sauf celles à exclure)
#    - Variables encodées (ordinales + nominales + binaires)

# Variables numériques à garder
variables_numeriques = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
variables_numeriques_a_garder = [col for col in variables_numeriques 
                                  if col not in colonnes_a_exclure]

print(f"\n✅ Variables NUMÉRIQUES à garder ({len(variables_numeriques_a_garder)}) :")
for i, col in enumerate(variables_numeriques_a_garder, 1):
    print(f"  {i:2d}. {col}")

# Créer X_numeriques
X_numeriques = df[variables_numeriques_a_garder]

print(f"\n📊 X_numeriques : {X_numeriques.shape}")

### 6.2. Fusion des features pour nouveau Dataset

In [ ]:
# Concaténer toutes les features
X = pd.concat([
    X_numeriques,           # Variables numériques originales
    df_encoded,             # Variables nominales encodées (OneHot)
], axis=1)

print(f"\n✅ DataFrame X créé avec succès !")
print(f"\n📊 Dimensions de X : {X.shape}")
print(f"   - {X.shape[0]} lignes (employés)")
print(f"   - {X.shape[1]} colonnes (features)")

display(X.columns.tolist()) 
print("="*80)


---

On a bien exclus les features catégorielles originales et la target

---

## 7. Matrice de Corrélation de Pearson

In [ ]:
# ============================================================================
# ÉTAPE 4.2 : VISUALISATION DE LA MATRICE DE CORRÉLATION
# ============================================================================

print("\n" + "="*80)
print("VISUALISATION DE LA MATRICE DE CORRÉLATION")
print("="*80)

correlation_matrix = X.corr(method='pearson')

# Créer la heatmap
plt.figure(figsize=(20, 16))
sns.heatmap(
    correlation_matrix, 
    annot=False,           # Pas d'annotation (trop de features)
    fmt='.2f', 
    cmap='RdYlBu_r',       # Rouge = corrélation positive, Bleu = négative
    center=0,              # Centre sur 0
    square=True, 
    linewidths=0.5,
    cbar_kws={"shrink": 0.8}
)
plt.title('Matrice de corrélation de Pearson (toutes les features)', 
          fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n💡 Interprétation des couleurs :")
print("   🔴 Rouge foncé   : Corrélation positive forte (proche de +1)")
print("   🔵 Bleu foncé    : Corrélation négative forte (proche de -1)")
print("   ⚪ Blanc/Jaune   : Pas de corrélation (proche de 0)")

In [ ]:
correlation_spearman = X.corr(method='spearman')
plt.figure(figsize=(20, 16))
sns.heatmap(
    correlation_spearman, 
    annot=True,           # Afficher les valeurs
    fmt='.2f',            # Format 2 décimales
    cmap='coolwarm',      # Palette de couleurs (bleu=négatif, rouge=positif)
    center=0,             # Centrer sur 0
    vmin=-1, vmax=1,      # Échelle de -1 à 1
    square=True,          # Cases carrées
    linewidths=0.5,       # Lignes entre cases
    cbar_kws={"shrink": 0.8}  # Taille de la barre de couleur
)
plt.title('Matrice de Corrélation de Spearman', fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.tight_layout()
plt.show()

### identification des corrélations fortes

In [ ]:
# ============================================================================
# ÉTAPE 4.3 : IDENTIFICATION DES CORRÉLATIONS FORTES
# ============================================================================

print("\n" + "="*80)
print("IDENTIFICATION DES PAIRES FORTEMENT CORRÉLÉES")
print("="*80)

# Seuil de corrélation à considérer comme "forte"
seuil = 0.7

# Trouver les paires de features fortement corrélées
correlations_fortes = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        
        if abs(corr_value) >= seuil:
            correlations_fortes.append({
                'Feature_1': correlation_matrix.columns[i],
                'Feature_2': correlation_matrix.columns[j],
                'Correlation': corr_value
            })

# Créer un DataFrame pour affichage
df_corr_fortes = pd.DataFrame(correlations_fortes)

if len(df_corr_fortes) > 0:
    # Trier par valeur absolue de corrélation (décroissant)
    df_corr_fortes['Abs_Correlation'] = df_corr_fortes['Correlation'].abs()
    df_corr_fortes = df_corr_fortes.sort_values('Abs_Correlation', ascending=False)
    
    print(f"\n⚠️  {len(df_corr_fortes)} paires de features avec corrélation ≥ {seuil} :\n")
    print("="*80)
    
    
    # Afficher le DataFrame
    display(df_corr_fortes[['Feature_1', 'Feature_2', 'Correlation']].reset_index(drop=True))
    
else:
    print(f"\n✅ Aucune paire de features avec corrélation ≥ {seuil}")
    print("   → Pas de multicolinéarité importante détectée !")

### analyse des corrélations moyennes

In [ ]:
# ============================================================================
# ÉTAPE 4.4 : CORRÉLATIONS MOYENNES (seuil 0.5 - 0.8)
# ============================================================================

print("\n" + "="*80)
print("CORRÉLATIONS MOYENNES (0.5 ≤ |r| < 0.8)")
print("="*80)

# Trouver les paires avec corrélation moyenne
seuil_min = 0.5
seuil_max = 0.8

correlations_moyennes = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        
        if seuil_min <= abs(corr_value) < seuil_max:
            correlations_moyennes.append({
                'Feature_1': correlation_matrix.columns[i],
                'Feature_2': correlation_matrix.columns[j],
                'Correlation': corr_value
            })

df_corr_moyennes = pd.DataFrame(correlations_moyennes)

if len(df_corr_moyennes) > 0:
    df_corr_moyennes['Abs_Correlation'] = df_corr_moyennes['Correlation'].abs()
    df_corr_moyennes = df_corr_moyennes.sort_values('Abs_Correlation', ascending=False)
    
    print(f"\n📊 {len(df_corr_moyennes)} paires avec corrélation moyenne ({seuil_min} ≤ |r| < {seuil_max})")
    
    # Afficher les 10 plus fortes
    print(f"\n🔝 Top 10 des corrélations moyennes les plus fortes :")
    print("="*80)
    display(df_corr_moyennes[['Feature_1', 'Feature_2', 'Correlation']].head(10).reset_index(drop=True))
else:
    print(f"\n✅ Aucune corrélation moyenne détectée")

## 8. Modélisation

In [ ]:
# ============================================================================
# ÉTAPE 3.1 : SÉPARATION TRAIN/TEST
# ============================================================================

from sklearn.model_selection import train_test_split
import pandas as pd

print("="*80)
print("ÉTAPE 3.1 : SÉPARATION TRAIN/TEST")
print("="*80)

# Vérifier qu'on a bien X et y
print(f"\n📊 Vérification des données :")
print(f"  • X (features) : {X.shape}")
print(f"  • y (target) : {y.shape}")
print(f"  • Même nombre de lignes ? {X.shape[0] == y.shape[0]} ✓")

# Distribution de y
print(f"\n📊 Distribution de la variable cible y :")
print(y.value_counts())
print(f"\n  • Classe 0 (Restés) : {(y==0).sum()} employés ({(y==0).sum()/len(y)*100:.2f}%)")
print(f"  • Classe 1 (Partis) : {(y==1).sum()} employés ({(y==1).sum()/len(y)*100:.2f}%)")

ratio_desequilibre = (y==0).sum() / (y==1).sum()
print(f"  • Ratio de déséquilibre : {ratio_desequilibre:.1f}:1")

# Séparation train/test
print(f"\n⏳ Séparation des données en cours...")
print(f"  • 80% pour l'entraînement (TRAIN)")
print(f"  • 20% pour le test (TEST)")
print(f"  • Stratification : OUI (pour garder les mêmes proportions)")

X_train, X_test, y_train, y_test = train_test_split(
    X,                      # Features
    y,                      # Target
    test_size=0.2,          # 20% pour le test
    random_state=42,        # Pour la reproductibilité
    stratify=y              # IMPORTANT : garde la même proportion 0/1
)

print("\n✅ Séparation effectuée avec succès !")

# Afficher les dimensions
print(f"\n📊 Dimensions des jeux de données :")
print(f"{'─'*80}")
print(f"  JEU D'ENTRAÎNEMENT (TRAIN) :")
print(f"    • X_train : {X_train.shape} → {X_train.shape[0]} employés, {X_train.shape[1]} features")
print(f"    • y_train : {y_train.shape}")
print(f"\n  JEU DE TEST (TEST) :")
print(f"    • X_test  : {X_test.shape} → {X_test.shape[0]} employés, {X_test.shape[1]} features")
print(f"    • y_test  : {y_test.shape}")

# Vérifier la stratification (très important !)
print(f"\n✅ Vérification de la STRATIFICATION :")
print(f"{'─'*80}")

print(f"\n  TRAIN :")
print(f"    • Restés (0) : {(y_train==0).sum()} ({(y_train==0).sum()/len(y_train)*100:.2f}%)")
print(f"    • Partis (1) : {(y_train==1).sum()} ({(y_train==1).sum()/len(y_train)*100:.2f}%)")

print(f"\n  TEST :")
print(f"    • Restés (0) : {(y_test==0).sum()} ({(y_test==0).sum()/len(y_test)*100:.2f}%)")
print(f"    • Partis (1) : {(y_test==1).sum()} ({(y_test==1).sum()/len(y_test)*100:.2f}%)")

print(f"\n  ORIGINAL :")
print(f"    • Restés (0) : {(y==0).sum()} ({(y==0).sum()/len(y)*100:.2f}%)")
print(f"    • Partis (1) : {(y==1).sum()} ({(y==1).sum()/len(y)*100:.2f}%)")

print(f"\n💡 Les proportions sont identiques dans train, test et original → Stratification OK !")

print(f"\n" + "="*80)
print("✅ ÉTAPE 3.1 TERMINÉE : Données prêtes pour la modélisation")
print("="*80)

In [ ]:
# ============================================================================
# PIPELINE COMPLET : TEST DE PLUSIEURS MODÈLES
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score
)
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("PIPELINE DE MODÉLISATION - TEST DE PLUSIEURS ALGORITHMES")
print("="*80)

# ============================================================================
# 1. DÉFINIR LES MODÈLES À TESTER
# ============================================================================

print("\n📋 Modèles à tester :")
print("─"*80)

modeles = {
    'Dummy (Baseline)': DummyClassifier(strategy='most_frequent', random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(max_depth=10, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
}

for i, nom in enumerate(modeles.keys(), 1):
    print(f"  {i}. {nom}")

print(f"\n✅ Total : {len(modeles)} modèles")

# ============================================================================
# 2. ENTRAÎNER ET ÉVALUER TOUS LES MODÈLES
# ============================================================================

print("\n" + "="*80)
print("ENTRAÎNEMENT ET ÉVALUATION DES MODÈLES")
print("="*80)

resultats = []
matrices_confusion = {}
predictions = {}

for nom_modele, modele in modeles.items():
    print(f"\n{'─'*80}")
    print(f"🔄 Modèle en cours : {nom_modele}")
    print(f"{'─'*80}")
    
    # Entraînement
    print("  ⏳ Entraînement...")
    modele.fit(X_train, y_train)
    print("  ✅ Entraîné")
    
    # Prédictions
    print("  ⏳ Prédictions...")
    y_pred_train = modele.predict(X_train)
    y_pred_test = modele.predict(X_test)
    
    # Probabilités (pour ROC-AUC)
    try:
        y_proba_test = modele.predict_proba(X_test)[:, 1]
    except:
        y_proba_test = y_pred_test  # Pour Dummy qui n'a pas predict_proba
    
    # Métriques sur TRAIN
    acc_train = accuracy_score(y_train, y_pred_train)
    
    # Métriques sur TEST
    acc_test = accuracy_score(y_test, y_pred_test)
    precision_test = precision_score(y_test, y_pred_test, zero_division=0)
    recall_test = recall_score(y_test, y_pred_test, zero_division=0)
    f1_test = f1_score(y_test, y_pred_test, zero_division=0)
    
    try:
        roc_auc_test = roc_auc_score(y_test, y_proba_test)
    except:
        roc_auc_test = 0.5  # Score aléatoire pour Dummy
    
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred_test)
    matrices_confusion[nom_modele] = cm
    predictions[nom_modele] = y_pred_test
    
    # Calculer overfitting
    overfitting = acc_train - acc_test
    
    # Stocker les résultats
    resultats.append({
        'Modèle': nom_modele,
        'Accuracy_train': acc_train,
        'Accuracy_test': acc_test,
        'Precision': precision_test,
        'Recall': recall_test,
        'F1-Score': f1_test,
        'ROC-AUC': roc_auc_test,
        'Overfitting': overfitting
    })
    
    # Afficher résumé
    print(f"  ✅ Terminé")
    print(f"     • Accuracy (test) : {acc_test:.4f}")
    print(f"     • Recall (test) : {recall_test:.4f}")
    print(f"     • F1-Score (test) : {f1_test:.4f}")

print("\n" + "="*80)
print("✅ TOUS LES MODÈLES ONT ÉTÉ ENTRAÎNÉS ET ÉVALUÉS")
print("="*80)

# ============================================================================
# 3. TABLEAU COMPARATIF DES PERFORMANCES
# ============================================================================

print("\n" + "="*80)
print("📊 TABLEAU COMPARATIF DES PERFORMANCES")
print("="*80)

df_resultats = pd.DataFrame(resultats)

# Trier par F1-Score (métrique équilibrée)
df_resultats = df_resultats.sort_values('F1-Score', ascending=False)

print("\n" + df_resultats.to_string(index=False))

# Identifier le meilleur modèle
meilleur_modele = df_resultats.iloc[0]['Modèle']
print(f"\n🏆 MEILLEUR MODÈLE (F1-Score) : {meilleur_modele}")

# Identifier le modèle avec le meilleur Recall
df_resultats_recall = df_resultats.sort_values('Recall', ascending=False)
meilleur_recall = df_resultats_recall.iloc[0]['Modèle']
print(f"🎯 MEILLEUR RECALL : {meilleur_recall} (Recall = {df_resultats_recall.iloc[0]['Recall']:.4f})")

# ============================================================================
# 4. VISUALISATIONS
# ============================================================================

print("\n" + "="*80)
print("📊 GÉNÉRATION DES VISUALISATIONS")
print("="*80)

# GRAPHIQUE 1 : Comparaison des métriques principales
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Accuracy
df_resultats.plot(x='Modèle', y='Accuracy_test', kind='barh', ax=axes[0, 0], 
                  color='steelblue', legend=False)
axes[0, 0].set_title('Accuracy (Test)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Score', fontsize=12)
axes[0, 0].grid(axis='x', alpha=0.3)

# Precision
df_resultats.plot(x='Modèle', y='Precision', kind='barh', ax=axes[0, 1], 
                  color='green', legend=False)
axes[0, 1].set_title('Precision (Test)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Score', fontsize=12)
axes[0, 1].grid(axis='x', alpha=0.3)

# Recall
df_resultats.plot(x='Modèle', y='Recall', kind='barh', ax=axes[1, 0], 
                  color='orange', legend=False)
axes[1, 0].set_title('Recall (Test) - Détection des départs', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Score', fontsize=12)
axes[1, 0].grid(axis='x', alpha=0.3)

# F1-Score
df_resultats.plot(x='Modèle', y='F1-Score', kind='barh', ax=axes[1, 1], 
                  color='purple', legend=False)
axes[1, 1].set_title('F1-Score (Test)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Score', fontsize=12)
axes[1, 1].grid(axis='x', alpha=0.3)

plt.suptitle('Comparaison des performances des modèles', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("✅ Graphique 1/3 : Métriques principales")

# GRAPHIQUE 2 : Overfitting (Train vs Test)
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(df_resultats))
width = 0.35

bars1 = ax.bar(x - width/2, df_resultats['Accuracy_train'], width, 
               label='Train', color='lightblue', edgecolor='black')
bars2 = ax.bar(x + width/2, df_resultats['Accuracy_test'], width, 
               label='Test', color='coral', edgecolor='black')

ax.set_xlabel('Modèles', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Overfitting : Accuracy Train vs Test', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(df_resultats['Modèle'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Graphique 2/3 : Overfitting (Train vs Test)")

# GRAPHIQUE 3 : Matrices de confusion
n_modeles = len(modeles)
n_cols = 3
n_rows = (n_modeles + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten()

for idx, (nom_modele, cm) in enumerate(matrices_confusion.items()):
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                ax=axes[idx], cbar=False,
                xticklabels=['Resté', 'Parti'],
                yticklabels=['Resté', 'Parti'])
    axes[idx].set_title(f'{nom_modele}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Prédit', fontsize=10)
    axes[idx].set_ylabel('Réel', fontsize=10)

# Masquer les subplots vides
for idx in range(len(modeles), len(axes)):
    axes[idx].axis('off')

plt.suptitle('Matrices de confusion - Tous les modèles', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("✅ Graphique 3/3 : Matrices de confusion")

# ============================================================================
# 5. ANALYSE DÉTAILLÉE DU MEILLEUR MODÈLE
# ============================================================================

print("\n" + "="*80)
print(f"🏆 ANALYSE DÉTAILLÉE : {meilleur_modele}")
print("="*80)

# Retrouver les prédictions du meilleur modèle
y_pred_best = predictions[meilleur_modele]
cm_best = matrices_confusion[meilleur_modele]

# Classification report
print("\n📋 Classification Report :")
print("─"*80)
print(classification_report(y_test, y_pred_best, 
                          target_names=['Resté (0)', 'Parti (1)'],
                          digits=4))

# Détail de la matrice de confusion
print("📊 Matrice de confusion détaillée :")
print("─"*80)
print(f"  TN (Restés bien prédits) : {cm_best[0,0]}")
print(f"  FP (Fausses alertes) : {cm_best[0,1]}")
print(f"  FN (Départs manqués) : {cm_best[1,0]}")
print(f"  TP (Départs détectés) : {cm_best[1,1]}")

# Interprétation métier
meilleur_row = df_resultats[df_resultats['Modèle'] == meilleur_modele].iloc[0]

print("\n" + "="*80)
print("💡 INTERPRÉTATION POUR TECHNOVA")
print("="*80)

print(f"\n✅ Le modèle {meilleur_modele} est le plus performant :")
print(f"   • F1-Score : {meilleur_row['F1-Score']:.2%}")
print(f"   • Recall : {meilleur_row['Recall']:.2%} → Détecte {meilleur_row['Recall']:.0%} des départs")
print(f"   • Precision : {meilleur_row['Precision']:.2%} → {meilleur_row['Precision']:.0%} des alertes sont vraies")

print(f"\n📊 Sur {(y_test==1).sum()} employés qui sont partis :")
print(f"   • {cm_best[1,1]} ont été DÉTECTÉS (TP)")
print(f"   • {cm_best[1,0]} ont été MANQUÉS (FN)")

print(f"\n⚠️  Overfitting : {meilleur_row['Overfitting']:.4f}")
if meilleur_row['Overfitting'] > 0.1:
    print("     → Attention : le modèle surappprend sur les données d'entraînement")
else:
    print("     → OK : le modèle généralise bien")

print("\n" + "="*80)
print("✅ PIPELINE TERMINÉ")
print("="*80)

# ============================================================================
# 6. SAUVEGARDE DES RÉSULTATS
# ============================================================================

print("\n💾 Sauvegarde des résultats...")

# Sauvegarder le DataFrame des résultats
df_resultats.to_csv('resultats_modeles.csv', index=False)
print("✅ Résultats sauvegardés dans 'resultats_modeles.csv'")

print("\n🎯 PROCHAINES ÉTAPES :")
print("  1. Optimiser le meilleur modèle (hyperparamètres)")
print("  2. Gérer le déséquilibre des classes (class_weight, SMOTE)")
print("  3. Utiliser SHAP pour l'interprétabilité")

### 8.1 optimisation avec le Recall

In [ ]:
# ============================================================================
# ÉTAPE 4.1 : OPTIMISATION AVEC CLASS_WEIGHT
# ============================================================================

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, recall_score, f1_score

print("="*80)
print("ÉTAPE 4.1 : OPTIMISATION - GESTION DU DÉSÉQUILIBRE")
print("="*80)

print("\n💡 Problème actuel :")
print("  • Le modèle est déséquilibré : 84% restés vs 16% partis")
print("  • Il favorise la classe majoritaire (Restés)")
print("  • Résultat : bon Accuracy mais mauvais Recall")

print("\n🎯 Solution : class_weight='balanced'")
print("  • Donne plus d'importance à la classe minoritaire (Partis)")
print("  • Force le modèle à mieux détecter les départs")

# ============================================================================
# 1. LOGISTIC REGRESSION avec class_weight
# ============================================================================

print("\n" + "─"*80)
print("🔄 Modèle 1 : Logistic Regression + class_weight='balanced'")
print("─"*80)

log_balanced = LogisticRegression(
    max_iter=1000, 
    random_state=42,
    class_weight='balanced'  # ← IMPORTANT
)

log_balanced.fit(X_train, y_train)
y_pred_log_balanced = log_balanced.predict(X_test)

print("\n📊 Résultats :")
print(classification_report(y_test, y_pred_log_balanced, 
                          target_names=['Resté', 'Parti']))

cm_log = confusion_matrix(y_test, y_pred_log_balanced)
print(f"\nMatrice de confusion :")
print(cm_log)
print(f"\n  TP (Départs détectés) : {cm_log[1,1]}/{(y_test==1).sum()}")
print(f"  FN (Départs manqués) : {cm_log[1,0]}/{(y_test==1).sum()}")

recall_log = recall_score(y_test, y_pred_log_balanced)
f1_log = f1_score(y_test, y_pred_log_balanced)
print(f"\n  → Recall : {recall_log:.2%}")
print(f"  → F1-Score : {f1_log:.2%}")

# ============================================================================
# 2. RANDOM FOREST avec class_weight
# ============================================================================

print("\n" + "─"*80)
print("🔄 Modèle 2 : Random Forest + class_weight='balanced'")
print("─"*80)

rf_balanced = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced',  # ← IMPORTANT
    n_jobs=-1
)

rf_balanced.fit(X_train, y_train)
y_pred_rf_balanced = rf_balanced.predict(X_test)

print("\n📊 Résultats :")
print(classification_report(y_test, y_pred_rf_balanced, 
                          target_names=['Resté', 'Parti']))

cm_rf = confusion_matrix(y_test, y_pred_rf_balanced)
print(f"\nMatrice de confusion :")
print(cm_rf)
print(f"\n  TP (Départs détectés) : {cm_rf[1,1]}/{(y_test==1).sum()}")
print(f"  FN (Départs manqués) : {cm_rf[1,0]}/{(y_test==1).sum()}")

recall_rf = recall_score(y_test, y_pred_rf_balanced)
f1_rf = f1_score(y_test, y_pred_rf_balanced)
print(f"\n  → Recall : {recall_rf:.2%}")
print(f"  → F1-Score : {f1_rf:.2%}")

# ============================================================================
# 3. COMPARAISON AVANT/APRÈS
# ============================================================================

print("\n" + "="*80)
print("📊 COMPARAISON : AVANT vs APRÈS class_weight")
print("="*80)

resultats_comparaison = pd.DataFrame([
    {
        'Modèle': 'Logistic Regression (sans class_weight)',
        'Recall': 0.3830,  # Votre résultat précédent
        'F1-Score': 0.5294,
        'TP': 18,
        'FN': 29
    },
    {
        'Modèle': 'Logistic Regression (AVEC class_weight)',
        'Recall': recall_log,
        'F1-Score': f1_log,
        'TP': cm_log[1,1],
        'FN': cm_log[1,0]
    },
    {
        'Modèle': 'Random Forest (AVEC class_weight)',
        'Recall': recall_rf,
        'F1-Score': f1_rf,
        'TP': cm_rf[1,1],
        'FN': cm_rf[1,0]
    }
])

print("\n" + resultats_comparaison.to_string(index=False))

# Identifier le meilleur
meilleur_idx = resultats_comparaison['Recall'].idxmax()
meilleur = resultats_comparaison.iloc[meilleur_idx]

print("\n" + "="*80)
print(f"🏆 MEILLEUR MODÈLE : {meilleur['Modèle']}")
print("="*80)
print(f"\n  • Recall : {meilleur['Recall']:.2%} → Détecte {meilleur['Recall']:.0%} des départs")
print(f"  • F1-Score : {meilleur['F1-Score']:.2%}")
print(f"  • Départs détectés : {int(meilleur['TP'])}/{(y_test==1).sum()}")
print(f"  • Départs manqués : {int(meilleur['FN'])}/{(y_test==1).sum()}")

gain_recall = meilleur['Recall'] - 0.3830
print(f"\n✅ Gain de Recall : +{gain_recall:.1%} par rapport au modèle initial")
print(f"   → {int(meilleur['TP']) - 18} départs supplémentaires détectés !")

print("\n" + "="*80)
print("✅ ÉTAPE 4.1 TERMINÉE")
print("="*80)

## 9. Les features importances

In [ ]:
# ============================================================================
# ÉTAPE 5 : FEATURE IMPORTANCE (VERSION SIMPLE - SANS SHAP)
# ============================================================================

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, f1_score, accuracy_score

print("="*80)
print("ÉTAPE 5 : INTERPRÉTABILITÉ - FEATURE IMPORTANCE")
print("="*80)

print("\n💡 Objectif : Identifier les CAUSES d'attrition chez TechNova")

# ============================================================================
# 1. MODÈLE FINAL
# ============================================================================

print("\n" + "─"*80)
print("🔄 Entraînement du modèle final")
print("─"*80)

modele_final = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced'
)

modele_final.fit(X_train, y_train)
print("✅ Modèle entraîné")

# Performance
y_pred_final = modele_final.predict(X_test)
print(f"\n📊 Performance (TEST) :")
print(f"  • Accuracy : {accuracy_score(y_test, y_pred_final):.2%}")
print(f"  • Recall : {recall_score(y_test, y_pred_final):.2%}")
print(f"  • F1-Score : {f1_score(y_test, y_pred_final):.2%}")

# ============================================================================
# 2. FEATURE IMPORTANCE (COEFFICIENTS)
# ============================================================================

print("\n" + "="*80)
print("📊 ANALYSE DES COEFFICIENTS DU MODÈLE")
print("="*80)

# Extraire les coefficients
coefficients = modele_final.coef_[0]

feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': coefficients,
    'Abs_Coefficient': np.abs(coefficients)
}).sort_values('Abs_Coefficient', ascending=False)

# TOP 20
print("\n🔝 TOP 20 features les plus importantes :")
print("─"*80)

for idx, (i, row) in enumerate(feature_importance.head(20).iterrows(), 1):
    feature = row['Feature']
    coef = row['Coefficient']
    
    # Interprétation
    if coef > 0:
        impact = "↗ AUGMENTE le risque de départ"
        emoji = "🔴"
    else:
        impact = "↘ DIMINUE le risque de départ (PROTÈGE)"
        emoji = "🟢"
    
    print(f"{idx:2d}. {feature[:50]:50s}")
    print(f"    Coefficient : {coef:+.4f}  {emoji} {impact}")
    print()

# ============================================================================
# 3. VISUALISATION
# ============================================================================

print("📊 Génération des graphiques...")

# GRAPHIQUE 1 : TOP 20 coefficients
fig, axes = plt.subplots(1, 2, figsize=(18, 10))

# Graphique 1a : Tous les coefficients (positifs et négatifs)
top_20 = feature_importance.head(20).sort_values('Coefficient')
colors = ['#e74c3c' if x > 0 else '#2ecc71' for x in top_20['Coefficient']]

axes[0].barh(range(len(top_20)), top_20['Coefficient'], 
             color=colors, edgecolor='black', linewidth=1)
axes[0].set_yticks(range(len(top_20)))
axes[0].set_yticklabels(top_20['Feature'], fontsize=9)
axes[0].set_xlabel('Coefficient', fontsize=12)
axes[0].set_title('TOP 20 Features - Impact sur l\'attrition\n🔴 Rouge = Favorise départs | 🟢 Vert = Protège contre départs', 
                  fontsize=12, fontweight='bold')
axes[0].axvline(x=0, color='black', linestyle='-', linewidth=1.5)
axes[0].grid(axis='x', alpha=0.3)

# Graphique 1b : Importance absolue
top_20_abs = feature_importance.head(20).sort_values('Abs_Coefficient')
axes[1].barh(range(len(top_20_abs)), top_20_abs['Abs_Coefficient'], 
             color='steelblue', edgecolor='black', linewidth=1)
axes[1].set_yticks(range(len(top_20_abs)))
axes[1].set_yticklabels(top_20_abs['Feature'], fontsize=9)
axes[1].set_xlabel('Importance absolue', fontsize=12)
axes[1].set_title('TOP 20 Features - Importance globale\n(sans tenir compte du sens)', 
                  fontsize=12, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Graphique 1/2 : Feature Importance")

# GRAPHIQUE 2 : Distribution des coefficients
fig, ax = plt.subplots(figsize=(12, 6))

ax.hist(coefficients, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Coefficient = 0')
ax.set_xlabel('Coefficient', fontsize=12)
ax.set_ylabel('Nombre de features', fontsize=12)
ax.set_title('Distribution des coefficients du modèle', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Graphique 2/2 : Distribution des coefficients")

# ============================================================================
# 4. INTERPRÉTATION MÉTIER
# ============================================================================

print("\n" + "="*80)
print("💼 INTERPRÉTATION MÉTIER POUR TECHNOVA")
print("="*80)

print("\n🎯 TOP 5 CAUSES D'ATTRITION :")
print("─"*80)

top_5 = feature_importance.head(5)

for idx, (i, row) in enumerate(top_5.iterrows(), 1):
    feature = row['Feature']
    coef = row['Coefficient']
    
    print(f"\n{idx}. {feature}")
    print(f"   Coefficient : {coef:+.4f}")
    
    if coef > 0:
        print(f"   💡 Plus cette variable augmente, plus le risque de départ AUGMENTE")
        print(f"   🎯 Action RH : Surveiller/améliorer ce facteur")
    else:
        print(f"   💡 Plus cette variable augmente, plus le risque de départ DIMINUE")
        print(f"   🎯 Action RH : Renforcer ce facteur protecteur")

# Facteurs de risque vs protecteurs
facteurs_risque = feature_importance[feature_importance['Coefficient'] > 0].head(5)
facteurs_protection = feature_importance[feature_importance['Coefficient'] < 0].head(5)

print("\n" + "="*80)
print("📋 SYNTHÈSE POUR LES RH")
print("="*80)

print("\n🔴 TOP 5 FACTEURS DE RISQUE (augmentent les départs) :")
for idx, (i, row) in enumerate(facteurs_risque.iterrows(), 1):
    print(f"  {idx}. {row['Feature'][:50]} (coef: {row['Coefficient']:+.4f})")

print("\n🟢 TOP 5 FACTEURS PROTECTEURS (réduisent les départs) :")
for idx, (i, row) in enumerate(facteurs_protection.iterrows(), 1):
    print(f"  {idx}. {row['Feature'][:50]} (coef: {row['Coefficient']:+.4f})")

print("\n" + "="*80)
print("✅ ÉTAPE 5 TERMINÉE")
print("="*80)

print("\n🎉 MISSION ACCOMPLIE !")
print("  ✅ Modèle optimisé : Recall de 68%")
print("  ✅ Causes d'attrition identifiées")
print("  ✅ Recommandations pour les RH")
print("\n⏭️  Il ne reste plus que la présentation PowerPoint !")